In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint



In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Keras module:", tf.keras)

TensorFlow version: 2.19.0
Keras module: <module 'tensorflow.keras' from 'd:\\Projects\\MoodMate\\paper_code\\paper_env\\Lib\\site-packages\\keras\\_tf_keras\\keras\\__init__.py'>


In [4]:
df=pd.read_csv(r"D:\Projects\MoodMate\paper_code\augmentation_code\final_extracted_features.csv")
df.head()

,0,1,2,3,4,5,6,7,8,9,...,76,77,78,79,80,81,82,83,84,label
0,-178.629257,51.506119,-41.428158,18.590761,-14.320748,-0.561634,-16.098753,-11.474614,-9.895323,0.881489,...,6.417930,6.499616,7.346619,8.597341,0.158534,0.114198,0.007965,0.029888,0.000000,angry
1,-354.181580,114.761452,29.720327,19.747877,-3.936052,3.929965,-4.114535,-10.306482,-2.094857,1.151795,...,3.098964,4.114218,5.604109,4.970121,0.069071,0.037858,0.000129,0.000689,55.434866,angry
2,-459.819641,92.469910,-25.202358,5.841247,-6.980287,-3.785758,-25.311193,-1.974278,-16.396502,-16.886845,...,12.345413,10.717521,12.245610,14.287747,0.090447,0.094145,0.000066,0.000215,49.316084,angry
3,-254.467255,57.603237,9.296393,8.734832,-3.518117,-1.730149,-2.974824,-9.274584,-3.085030,-13.884545,...,4.823880,6.712316,5.028618,5.914686,0.223901,0.207565,0.003188,0.012238,0.000000,angry
4,-299.528473,57.804699,6.727249,16.468393,0.942798,-6.469413,-2.439889,-3.310119,-6.644002,-9.503429,...,8.389625,6.659488,5.193701,4.733158,0.160334,0.139378,0.007396,0.038400,49.595356,angry


In [5]:
X=df.drop(columns=['label']).values
y=df['label']

In [6]:
mean_imputer=SimpleImputer(strategy='mean')
X[np.isinf(X)]=np.nan
X[:,84:85]=mean_imputer.fit_transform(X[:,84:85])

In [7]:
scaling_model=joblib.load(r'D:\Projects\MoodMate\paper_code\models\scaler.pkl')
label_encoding_model=joblib.load(r"D:\Projects\MoodMate\paper_code\models\label_encoder.pkl")


d:\Projects\MoodMate\paper_code\paper_env\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.0 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Projects\MoodMate\paper_code\paper_env\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.3.0 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
X_scaled=scaling_model.transform(X)
y_encoded=label_encoding_model.transform(y)

In [9]:
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 5, 17))
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_encoded, test_size=0.2, random_state=42)

In [10]:
y_train_class = np.argmax(y_train, axis=1)  # shape: (39600,)
y_test_class = np.argmax(y_test, axis=1)

In [11]:
label_encoding_model.classes_[0]
np.where(label_encoding_model.classes_ == 'sad')[0][0]
inds=np.flatnonzero(label_encoding_model.classes_ == 'sad')
if inds.size == 0:
    raise ValueError(f"'{'sad'}' not found in encoder classes: {label_encoding_model.classes_}")
int(inds[0])

5

In [12]:
y_train

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]])

In [13]:
y_train_class = np.argmax(y_train, axis=1)
y_train_bin = (y_train_class == 0).astype(int)
y_train_bin

array([1, 0, 0, ..., 0, 1, 0])

In [14]:
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad']
models = {}  # to store trained models

# Convert one-hot encoded y_train/y_test to class labels
y_train_class = np.argmax(y_train, axis=1)
y_test_class = np.argmax(y_test, axis=1)

for i, emotion in enumerate(emotion_labels):
    print(f"\nTraining model for '{emotion}'...")
    print(f"Class index: {i}")
    
    # Convert y to binary: 1 if sample belongs to class `i`, else 0
    y_train_bin = (y_train_class == i).astype(int)
    y_test_bin = (y_test_class == i).astype(int)

    print("First label (one-hot):", y_train[0])
    print("First label (class):", y_train_class[0])
    print("First label (binary for this emotion):", y_train_bin[0])

    # Now you can pass X_train and y_train_bin to the model for this emotion

    break  # remove break to loop over all emotions



Training model for 'angry'...
Class index: 0
First label (one-hot): [1 0 0 0 0 0]
First label (class): 0
First label (binary for this emotion): 1
